### Ejercicio 3: Introducción a la optimización

Una empresa produce dos productos (x e y) con un beneficio neto de 40 y 30 $/kg respectivamente. Sin embargo, por restricciones de demanda, solo se pueden producir como mucho 40 kg de x en cada lote. Por capacidad de la planta solo se pueden producir 80 kg de productos totales. El almacenamiento de los productos también es una limitación, no pudiéndose almacenar más de 100 m3 de productos (densidad de 2.1 m3/kg y 1.7 m3/kg de x e y respectivamente). Establezca cuál es la cantidad óptima de x e y que se pueden producir

$$
\begin{align}
	&\text{max }  && z=40x+30y &&\\
	&\text{s.t. }  && x    \le  40\\
	&  && x+y \le  80\\
    &  && 2.1x+1.7y \le100 \\
    & && x,y \geq  0  \\
\end{align}
$$

#### El primer paso será instalar Pyomo y dos solvers (ipopt y glpk)

Esto lo ejecutamos desde el terminal de Anaconda:

conda install -c conda-forge pyomo

conda install -c conda-forge ipopt=3.11.1 glpk

In [1]:
!pyomo help --solvers


Pyomo Solvers and Solver Managers
---------------------------------
Pyomo uses 'solver managers' to execute 'solvers' that perform
optimization and other forms of model analysis.  A solver directly
executes an optimizer, typically using an executable found on the
user's PATH environment.  Solver managers support a flexible mechanism
for asyncronously executing solvers either locally or remotely.  The
following solver managers are available in Pyomo:

    neos       Asynchronously execute solvers on the NEOS server
    serial     Synchronously execute solvers locally

If no solver manager is specified, Pyomo uses the serial solver
manager to execute solvers locally.  The neos solver manager is used
to execute solvers on the NEOS optimization server.


Serial Solver Interfaces
------------------------
The serial manager supports the following solver interfaces:

    appsi_cbc                   Automated persistent interface to Cbc
    appsi_cplex                 Automated persistent int

#### Importamos Pyomo

In [2]:
from pyomo.environ import *

#### Creamos un problema concreto, es decir, un objeto de la clase problema que ya contenga parámetros

In [3]:
model = ConcreteModel()

#### Definimos las variables como no negativas

In [4]:
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

#### Definimos la función objetivo

In [5]:
model.profit = Objective(expr = 40*model.x + 30*model.y,
                         sense=maximize)

#### Definimos la restricciones

In [6]:
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2.1*model.x + 1.7*model.y <= 100)

In [7]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  40.0 :  None : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 9.41176470588235 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 2.1*x + 1.7*y : 100.0 :   True

6 Declarations: x y 

In [8]:
model.x.value

40.0

In [9]:
model.y.value

9.41176470588235

In [10]:
model.profit.expr()

1882.3529411764705